# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

In [3]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
import os

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [2]:
#TODO: Fetch and upload the data to AWS S3

# Command to download and unzip data
# !wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
# !unzip dogImages.zip

import sagemaker
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from torchvision.datasets import CIFAR10
from torchvision import transforms
from sagemaker.pytorch import PyTorch
import gzip 
import numpy as np
import random
import os

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/pytorch-cifar"
role = sagemaker.get_execution_role()

local_dir = 'data'
CIFAR10.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/CIFAR10/"]
CIFAR10(
    local_dir,
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor()]
    )
)


inputs = sagemaker_session.upload_data(path="data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))



Files already downloaded and verified
input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-732873316855/sagemaker/pytorch-cifar


## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [4]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch-size": CategoricalParameter([32, 64, 128, 256, 512]),
    "epochs": IntegerParameter(2, 4)
}

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

In [18]:
estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.large"
)



tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type=objective_type,
)


In [14]:
tuner.fit({"training": inputs}, wait = True)


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


........................................................................................................................................................................................................................................................................................!


In [16]:
# TODO: Get the best estimators and the best HPs
best_estimator = tuner.best_estimator() 

best_estimator.hyperparameters()


2022-12-02 14:21:17 Starting - Found matching resource for reuse
2022-12-02 14:21:17 Downloading - Downloading input data
2022-12-02 14:21:17 Training - Training image download completed. Training in progress.
2022-12-02 14:21:17 Uploading - Uploading generated training model
2022-12-02 14:21:17 Completed - Resource retained for reuse


{'_tuning_objective_metric': '"average test loss"',
 'batch-size': '"64"',
 'epochs': '3',
 'lr': '0.002188635544383642',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch-training-2022-12-02-14-04-00-622"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-732873316855/pytorch-training-2022-12-02-14-04-00-622/source/sourcedir.tar.gz"'}

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [5]:
#Set up debugging and profiling rules and hooks
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

from sagemaker.debugger import (
    Rule,
    DebuggerHookConfig,
    rule_configs,
)

from sagemaker.debugger import Rule, ProfilerRule, rule_configs

rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

In [6]:
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

hook_config = DebuggerHookConfig(
    
    hook_parameters = {"train.save_interval": "100", "eval.save_interval": "10"}
)

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)

In [7]:
hyperparameters = {
    "batch-size": 64, #From Previous tuner
    "epochs": 3, #From previous tuner
    "lr":  0.002188635544383642 #From Previous tuner
}

In [8]:
Create and fit an estimator

import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

estimator = PyTorch(
    entry_point="train_model.py",
    base_job_name="sagemaker-script-mode",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.large",
    hyperparameters = hyperparameters,
    framework_version="1.8",
    py_version="py36",
    ## Debugger parameters
    rules=rules,
    debugger_hook_config=hook_config,
    ## Profiler parameters
    profiler_config=profiler_config,
    env={'MMS_DEFAULT_RESPONSE_TIMEOUT': '1000'}
)

In [9]:
estimator.fit({"training":inputs}, wait = True)

2022-12-03 15:37:28 Starting - Starting the training job...
2022-12-03 15:37:54 Starting - Preparing the instances for trainingVanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LossNotDecreasing: InProgress
LowGPUUtilization: InProgress
ProfilerReport: InProgress
......
2022-12-03 15:38:55 Downloading - Downloading input data......
2022-12-03 15:39:55 Training - Downloading the training image...
2022-12-03 15:40:23 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-12-03 15:40:25,547 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-12-03 15:40:25,550 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-03 15:40:25,559 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022

In [10]:
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

session = boto3.session.Session()
region = session.region_name

training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

trial = create_trial(estimator.latest_job_debugger_artifacts_path())

[2022-12-03 15:48:11.506 pytorch-1-8-gpu-py36-ml-m5-large-f6c3d7d670ee91b6cbdc84ad69d6:27 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
Training jobname: sagemaker-script-mode-2022-12-03-15-37-27-955
Region: us-east-1
[2022-12-03 15:48:11.526 pytorch-1-8-gpu-py36-ml-m5-large-f6c3d7d670ee91b6cbdc84ad69d6:27 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-east-1-732873316855/sagemaker-script-mode-2022-12-03-15-37-27-955/debug-output


In [11]:
print(trial.tensor_names())
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN)))
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL)))

[2022-12-03 15:48:11.895 pytorch-1-8-gpu-py36-ml-m5-large-f6c3d7d670ee91b6cbdc84ad69d6:27 INFO trial.py:198] Training has ended, will refresh one final time in 1 sec.
[2022-12-03 15:48:12.911 pytorch-1-8-gpu-py36-ml-m5-large-f6c3d7d670ee91b6cbdc84ad69d6:27 INFO trial.py:210] Loaded all steps
['CrossEntropyLoss_output_0', 'gradient/ResNet_fc.0.bias', 'gradient/ResNet_fc.0.weight', 'layer1.0.relu_input_0', 'layer1.0.relu_input_1', 'layer1.1.relu_input_0', 'layer1.1.relu_input_1', 'layer2.0.relu_input_0', 'layer2.0.relu_input_1', 'layer2.1.relu_input_0', 'layer2.1.relu_input_1', 'layer3.0.relu_input_0', 'layer3.0.relu_input_1', 'layer3.1.relu_input_0', 'layer3.1.relu_input_1', 'layer4.0.relu_input_0', 'layer4.0.relu_input_1', 'layer4.1.relu_input_0', 'layer4.1.relu_input_1', 'relu_input_0']
2
4


In [12]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

ProfilerConfig:{'S3OutputPath': 's3://sagemaker-us-east-1-732873316855/', 'ProfilingIntervalInMilliseconds': 500, 'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "MetricsRegex": ".*", }', 'DetailedProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'FileOpenFailThreshold': '50', 'HorovodProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'LocalPath': '/opt/ml/output/profiler', 'PythonProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }', 'RotateFileCloseIntervalInSeconds': '60', 'RotateMaxFileSizeInBytes': '10485760', 'SMDataParallelProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }'}}
s3 path:s3://sagemaker-us-east-1-732873316855/sagemaker-script-mode-2022-12-03-15-37-27-955/profiler-output


Profiler data from system is available


In [13]:
system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()


In [14]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)


[2022-12-03 15:48:13.468 pytorch-1-8-gpu-py36-ml-m5-large-f6c3d7d670ee91b6cbdc84ad69d6:27 INFO metrics_reader_base.py:134] Getting 9 event files
select events:['total']
select dimensions:['CPU', 'GPU']
filtered_events:{'total'}
filtered_dimensions:{'CPUUtilization-nodeid:algo-1'}


In [15]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

You will find the profiler report in s3://sagemaker-us-east-1-732873316855/sagemaker-script-mode-2022-12-03-15-37-27-955/rule-output


In [16]:
! aws s3 ls {rule_output_path} --recursive


2022-12-03 15:47:52     386566 sagemaker-script-mode-2022-12-03-15-37-27-955/rule-output/ProfilerReport/profiler-output/profiler-report.html
2022-12-03 15:47:51     236598 sagemaker-script-mode-2022-12-03-15-37-27-955/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2022-12-03 15:47:46        191 sagemaker-script-mode-2022-12-03-15-37-27-955/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2022-12-03 15:47:46        199 sagemaker-script-mode-2022-12-03-15-37-27-955/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2022-12-03 15:47:46       1992 sagemaker-script-mode-2022-12-03-15-37-27-955/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2022-12-03 15:47:46        127 sagemaker-script-mode-2022-12-03-15-37-27-955/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
2022-12-03 15:47:46        198 sagemaker-script-mode-2022-12-03-15-37-27-955/rule-output/Profile

In [17]:
! aws s3 cp {rule_output_path} ./ --recursive


download: s3://sagemaker-us-east-1-732873316855/sagemaker-script-mode-2022-12-03-15-37-27-955/rule-output/ProfilerReport/profiler-output/profiler-report.html to ProfilerReport/profiler-output/profiler-report.html
download: s3://sagemaker-us-east-1-732873316855/sagemaker-script-mode-2022-12-03-15-37-27-955/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json to ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
download: s3://sagemaker-us-east-1-732873316855/sagemaker-script-mode-2022-12-03-15-37-27-955/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb to ProfilerReport/profiler-output/profiler-report.ipynb
download: s3://sagemaker-us-east-1-732873316855/sagemaker-script-mode-2022-12-03-15-37-27-955/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json to ProfilerReport/profiler-output/profiler-reports/BatchSize.json
download: s3://sagemaker-us-east-1-732873316855/sagemaker-script-mode-2022-12-03-15-37-27-955/

In [18]:
# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

In [19]:
import os

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

In [20]:
profiler_report_name

'ProfilerReport'

In [ ]:
import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

## Model Deploying

In [22]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy(initial_instance_count=1, instance_type="ml.m5.large")  

INFO:sagemaker:Creating model with name: sagemaker-script-mode-2022-12-03-15-48-16-101
INFO:sagemaker:Creating endpoint with name sagemaker-script-mode-2022-12-03-15-48-16-101


-----!

In [23]:
predictor.endpoint_name

'sagemaker-script-mode-2022-12-03-15-48-16-101'

In [24]:
import gzip 
import numpy as np
import random
import os

file = 'data/cifar-10-batches-py/data_batch_1'
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        Data = pickle.load(fo, encoding='bytes')
    return Data

Data_extracted=unpickle(file)
data=np.reshape(Data_extracted[b'data'][0], (3, 32, 32))

In [25]:
#image from the training dataset
Imgs_train=unpickle('data/cifar-10-batches-py/data_batch_1') 

In [26]:
#Information about the picked Image
n=0
print(Imgs_train[b'labels'][n])
print(Imgs_train[b'data'][n].shape)
print(Imgs_train[b'filenames'][n])
Img_train=np.reshape(Imgs_train[b'data'][n], (3, 32, 32))
print(Img_train)

6
(3072,)
b'leptodactylus_pentadactylus_s_000004.png'
[[[ 59  43  50 ... 158 152 148]
  [ 16   0  18 ... 123 119 122]
  [ 25  16  49 ... 118 120 109]
  ...
  [208 201 198 ... 160  56  53]
  [180 173 186 ... 184  97  83]
  [177 168 179 ... 216 151 123]]

 [[ 62  46  48 ... 132 125 124]
  [ 20   0   8 ...  88  83  87]
  [ 24   7  27 ...  84  84  73]
  ...
  [170 153 161 ... 133  31  34]
  [139 123 144 ... 148  62  53]
  [144 129 142 ... 184 118  92]]

 [[ 63  45  43 ... 108 102 103]
  [ 20   0   0 ...  55  50  57]
  [ 21   0   8 ...  50  50  42]
  ...
  [ 96  34  26 ...  70   7  20]
  [ 96  42  30 ...  94  34  34]
  [116  94  87 ... 140  84  72]]]


In [27]:
data_batch_index=np.expand_dims(Img_train, axis=0)
data_batch_index.shape


(1, 3, 32, 32)

In [ ]:
Run an prediction on the endpoint
import torch
response = predictor.predict(torch.Tensor(data_batch_index)) #convert array to a byte tensor


# Query the endpoint
print(response)

In [ ]:
m = torch.nn.Softmax(dim=1)


In [ ]:
m(torch.Tensor(response))


In [ ]:
#Information about the picked Image
n=1
print(Imgs_train[b'labels'][n])
print(Imgs_train[b'data'][n].shape)
print(Imgs_train[b'filenames'][n])
Img_train2=np.reshape(Imgs_train[b'data'][n], (3, 32, 32))
print(Img_train2)

In [ ]:
data_batch_index2=np.expand_dims(Img_train2, axis=0)
data_batch_index2.shape

In [ ]:
# Run an prediction on the endpoint
response2 = predictor.predict(torch.Tensor(data_batch_index2)) #convert array to a byte tensor

# Query the endpoint
print(response2)

In [ ]:
m = torch.nn.Softmax(dim=1)


In [ ]:
m(torch.Tensor(response2))


In [29]:
# Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-script-mode-2022-12-03-15-48-16-101
INFO:sagemaker:Deleting endpoint with name: sagemaker-script-mode-2022-12-03-15-48-16-101
